# Fetch Rewards: ML Apprentice Take-Home Overview

### Please see write-up for rationale + technical justification

In [1]:
# # Package installs -> Uncomment when running the first time!

# ! pip install --quiet torch
# ! pip install --quiet numpy
# ! pip install --quiet transformers
# # ! pip install --quiet hf_xet
# ! pip install --quiet scikit-learn
# # ! pip install --user --quiet ipywidgets widgetsnbextension

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer
import numpy as np
from torch import optim
from sklearn.metrics import classification_report

c:\Users\nibha\anaconda3\envs\new_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Task 1: Sentence Transformer Implementation

In [3]:
def mean_pooling(token_embeddings, attention_mask):
    """
    Compute mean pooling of token embeddings, ignoring padding tokens.
    
    Args:
        token_embeddings: Tensor of shape [batch_size, seq_len, hidden_size]
        attention_mask: Tensor of shape [batch_size, seq_len] with 1s for tokens and 0s for padding
        
    Returns:
        Tensor of shape [batch_size, hidden_size] with sentence embeddings
    """
    # Convert attention mask to float for multiplication
    mask = attention_mask.unsqueeze(-1).float()
    
    # Sum the embeddings of real tokens (multiply by mask to zero out padding)
    sum_embeddings = torch.sum(token_embeddings * mask, dim=1)
    
    # Count the number of real tokens per sentence
    token_counts = torch.sum(attention_mask, dim=1, keepdim=True).float()
    
    # Compute mean by dividing sum by count
    sentence_embeddings = sum_embeddings / token_counts
    
    return sentence_embeddings


def max_pooling(token_embeddings, attention_mask):
    """
    Compute max pooling of token embeddings, ignoring padding tokens.
    
    Args:
        token_embeddings: Tensor of shape [batch_size, seq_len, hidden_size]
        attention_mask: Tensor of shape [batch_size, seq_len] with 1s for tokens and 0s for padding
        
    Returns:
        Tensor of shape [batch_size, hidden_size] with sentence embeddings
    """
    # Create a mask for padding tokens (0s for padding, 1s for real tokens)
    mask = attention_mask.unsqueeze(-1).expand_as(token_embeddings).float()
    
    # Replace padding token embeddings with large negative values
    # This ensures they won't be selected during max operation
    masked_embeddings = token_embeddings.clone()
    masked_embeddings[mask == 0] = -1e9
    
    # Take max over sequence dimension (dim=1)
    sentence_embeddings = torch.max(masked_embeddings, dim=1)[0]
    
    return sentence_embeddings


In [4]:
class SentenceTransformer(nn.Module):
    def __init__(self, model_name='bert-base-uncased', embedding_dim=768, pooling_strategy='mean', normalize_embeddings=True):
        """
        Initialize the Sentence Transformer model.
        
        Args:
            model_name: Pre-trained transformer model name
            embedding_dim: Dimension of embeddings from the transformer model
            pooling_strategy: Strategy to pool token embeddings into sentence embedding
            projection_dim: If provided, project embeddings to this dimension
            normalize_embeddings: Whether to normalize final embeddings
        """
        super(SentenceTransformer, self).__init__()
        
        # Load pre-trained transformer model
        self.transformer = BertModel.from_pretrained(model_name)
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.embedding_dim = embedding_dim
        
        # Configure pooling strategy
        self.pooling_strategy = pooling_strategy
            
        # Whether to normalize final embeddings
        self.normalize_embeddings = normalize_embeddings
        
        
    def forward(self, input_ids, attention_mask):
        """Forward pass through the model"""
        # Get transformer outputs
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get embeddings from the transformer output
        token_embeddings = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]
        
        # Apply pooling strategy
        if self.pooling_strategy == 'cls':
            # Use [CLS] token embedding
            sentence_embedding = token_embeddings[:, 0, :]
        elif self.pooling_strategy == 'mean':
            # Mean pooling - take mean of all BERT embeddings for a sentence
            sentence_embedding = mean_pooling(token_embeddings, attention_mask)
        elif self.pooling_strategy == 'max':
            # Max pooling - take max of all BERT embeddings for a sentence
            sentence_embedding = max_pooling(token_embeddings, attention_mask)
        else:
            raise ValueError(f"Unknown pooling strategy: {self.pooling_strategy}")
        
        # Normalize embeddings so that similar sentences can be compared via cosine similarity
        if self.normalize_embeddings:
            sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)
            
        return sentence_embedding

    
    def encode_single(self, sentence, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Encode a single sentence into an embedding
        
        Args:
            sentence: String sentence to encode
            device: Device to use for computation
        
        Returns:
            numpy array of sentence embedding
        """
        self.to(device)
        self.eval()
        
        # Tokenize the sentence
        inputs = self.tokenizer(sentence, padding=True, truncation=True, 
                               return_tensors="pt", max_length=512)
        
        # Move inputs to device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Get embedding
        with torch.no_grad():
            embedding = self.forward(inputs['input_ids'], inputs['attention_mask'])
        
        # Move embedding to CPU and convert to numpy
        return embedding.detach().cpu().numpy()[0]  # Get the first (and only) item in batch


    def encode(self, sentences, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Encode a list of sentences into embeddings, processing one sentence at a time
        
        Args:
            sentences: List of sentences to encode
            device: Device to use for computation
        
        Returns:
            numpy array of sentence embeddings
        """
        self.to(device)
        self.eval()
        
        all_embeddings = []
        
        # Process sentences one at a time
        for sentence in sentences:
            embedding = self.encode_single(sentence, device)
            all_embeddings.append(embedding)
        
        # Stack all embeddings
        all_embeddings = np.vstack(all_embeddings)
        
        return all_embeddings


In [5]:
def main():
    """Generate the fixed-length sentence embeddings"""
    # Initialize model with different configurations
    models = {
        "BERT-base with mean pooling": SentenceTransformer(
            model_name='bert-base-uncased', 
            pooling_strategy='mean',
            normalize_embeddings=True
        ),
        "BERT-base with CLS pooling": SentenceTransformer(
            model_name='bert-base-uncased', 
            pooling_strategy='cls',
            normalize_embeddings=True
        ),
    }
        
    # Sample sentences
    sample_sentences = [
        "The new smartphone features an AI-powered camera and improved battery life.",
        "The team scored a last-minute goal to win the championship.",
        "The senator proposed a new bill to address climate change.",
        "The movie received excellent reviews from critics and audiences alike.",
        "Regular exercise and a balanced diet are essential for maintaining good health.",
        "The software update includes security patches and performance improvements.",
        "The athlete broke the world record in the 100-meter sprint.",
        "The president will deliver a speech on economic policy tomorrow.",
        "The concert tickets sold out within minutes of going on sale.",
        "The study found a strong correlation between sleep quality and cognitive function."
    ][:3]
    
    # Test each model configuration
    for model_name, model in models.items():
        print(f"\nTesting: {model_name}")
        
        # Encode sentences
        embeddings = model.encode(sample_sentences)
        
        # Print embedding dimensions
        print(f"Embedding shape: {embeddings.shape}, Number of sentences: {len(sample_sentences)}")
        
        print("\nSample embeddings (first 5 dimensions):")
        for i, sentence in enumerate(sample_sentences):
            print(f"Sentence: {sentence}")
            print(f"Embedding: {embeddings[i][:5]}...")
            print()


if __name__ == "__main__":
    main()
    


Testing: BERT-base with mean pooling
Embedding shape: (3, 768), Number of sentences: 3

Sample embeddings (first 5 dimensions):
Sentence: The new smartphone features an AI-powered camera and improved battery life.
Embedding: [-0.01046181 -0.00928617  0.05656587 -0.00583826  0.03553018]...

Sentence: The team scored a last-minute goal to win the championship.
Embedding: [-0.0263459  -0.05063563  0.02050924 -0.02016847  0.02163503]...

Sentence: The senator proposed a new bill to address climate change.
Embedding: [-0.00652519 -0.04245586 -0.03979944  0.00203729 -0.00385006]...


Testing: BERT-base with CLS pooling
Embedding shape: (3, 768), Number of sentences: 3

Sample embeddings (first 5 dimensions):
Sentence: The new smartphone features an AI-powered camera and improved battery life.
Embedding: [-0.02002563 -0.02972431  0.01177708 -0.01269144 -0.00239184]...

Sentence: The team scored a last-minute goal to win the championship.
Embedding: [-0.03604771 -0.02874261 -0.0050998  -0.025

## Task 2: Multi-Task Learning Expansion

### Task A: Sentence Classification

In [6]:
class SentenceClassifier(nn.Module):
    def __init__(self, sentence_transformer, num_classes, hidden_layers=None, freeze_transformer=True):
        """
        Initialize a classifier that uses sentence embeddings.
        
        Args:
            sentence_transformer: SentenceTransformer model for creating embeddings
            num_classes: Number of output classes for classification/sentiment analysis
            hidden_layers: List of hidden layer sizes (if None, only a single classification layer is used)
            freeze_transformer: Whether to freeze the transformer part during training
        """
        super(SentenceClassifier, self).__init__()
        
        self.sentence_transformer = sentence_transformer
        
        # Freeze transformer parameters if specified
        if freeze_transformer:
            for param in self.sentence_transformer.transformer.parameters():
                param.requires_grad = False
                
        # Get embedding dimension
        embedding_dim = self.sentence_transformer.embedding_dim
        
        # Create classifier layers
        if hidden_layers is None:
            # Simple linear classifier
            self.classifier = nn.Linear(embedding_dim, num_classes)
        else:
            # Multi-layer classifier
            layers = []
            input_dim = embedding_dim
            
            # Add hidden layers
            for hidden_dim in hidden_layers:
                layers.append(nn.Linear(input_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(0.1))
                input_dim = hidden_dim
                
            # Add output layer
            layers.append(nn.Linear(input_dim, num_classes))
            
            self.classifier = nn.Sequential(*layers)
    
    def forward(self, input_ids, attention_mask):
        """Forward pass through the model"""
        # Get sentence embeddings
        embeddings = self.sentence_transformer(input_ids, attention_mask)
        
        # Apply classifier
        logits = self.classifier(embeddings)
        
        return logits
    
    def classify(self, sentences, class_names=None, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Classify a list of sentences
        
        Args:
            sentences: List of sentences to classify
            class_names: List of class names (if None, returns class indices)
            device: Device to use for computation
            
        Returns:
            List of predicted classes
        """
        self.to(device)
        self.eval()
        
        all_predictions = []
        
        # Process sentences one at a time
        for sentence in sentences:
            # Tokenize the sentence
            inputs = self.sentence_transformer.tokenizer(sentence, padding=True, truncation=True, 
                                                        return_tensors="pt", max_length=512)
            
            # Move inputs to device
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # Get predictions
            with torch.no_grad():
                logits = self.forward(inputs['input_ids'], inputs['attention_mask'])
                prediction = torch.argmax(logits, dim=1).item()
            
            # Convert to class name if provided
            if class_names is not None:
                prediction = class_names[prediction]
                
            all_predictions.append(prediction)
        
        return all_predictions
    
    def pred_probs(self, sentences, device='cuda' if torch.cuda.is_available() else 'cpu'):
        """
        Get class probabilities for a list of sentences
        
        Args:
            sentences: List of sentences to classify
            device: Device to use for computation
            
        Returns:
            Numpy array of class probabilities
        """
        self.to(device)
        self.eval()
        
        all_probs = []
        
        # Process sentences one at a time
        for sentence in sentences:
            # Tokenize the sentence
            inputs = self.sentence_transformer.tokenizer(sentence, padding=True, truncation=True, 
                                                        return_tensors="pt", max_length=512)
            
            # Move inputs to device
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # Get predictions
            with torch.no_grad():
                logits = self.forward(inputs['input_ids'], inputs['attention_mask'])
                probs = F.softmax(logits, dim=1)
            
            all_probs.append(probs.cpu().numpy()[0])
        
        return np.array(all_probs)


In [7]:

def sentence_classification():
    """Run the inference for the classification model with sentence embedding backbone"""
    # Define a sample classification task
    class_names = ["Technology", "Sports", "Politics", "Entertainment", "Health"]
    
    # Sample data
    sample_sentences = [
        "The new smartphone features an AI-powered camera and improved battery life.",
        "The team scored a last-minute goal to win the championship.",
        "The senator proposed a new bill to address climate change.",
        "The movie received excellent reviews from critics and audiences alike.",
        "Regular exercise and a balanced diet are essential for maintaining good health.",
        "The software update includes security patches and performance improvements.",
        "The athlete broke the world record in the 100-meter sprint.",
        "The president will deliver a speech on economic policy tomorrow.",
        "The concert tickets sold out within minutes of going on sale.",
        "The study found a strong correlation between sleep quality and cognitive function."
    ]
    
    # Sample labels (indices of class_names)
    sample_labels = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
    
    # Initialize the sentence transformer
    sentence_transformer = SentenceTransformer(
        model_name='bert-base-uncased',
        pooling_strategy='cls',
        normalize_embeddings=True
    )
    
    # Initialize the classifier
    classifier = SentenceClassifier(
        sentence_transformer=sentence_transformer,
        num_classes=len(class_names),
        hidden_layers=[256, 128],  # Two hidden layers
        freeze_transformer=True    # Freeze transformer weights
    )

    # Make predictions
    print("\nPredicting classes...")
    predictions = classifier.classify(sample_sentences[:2], class_names=class_names)
    
    # Print predictions
    print("\nPredictions:")
    for i, (sentence, true_class, pred_class) in enumerate(zip(sample_sentences[:2], 
                                                            [class_names[i] for i in sample_labels[:2]], 
                                                            predictions)):
        print(f"Sentence: {sentence}")
        print(f"True class: {true_class}")
        print(f"Predicted class: {pred_class}")
        print()
    
    # Get classification probabilities
    print("\nClass probabilities:")
    probs = classifier.pred_probs(sample_sentences[:2])  # Just show first two for brevity
    for i, (sentence, prob) in enumerate(zip(sample_sentences[:2], probs)):
        print(f"Sentence: {sentence}")
        for j, class_name in enumerate(class_names):
            print(f"  {class_name}: {prob[j]:.4f}")
        print()

if __name__ == "__main__":
    sentence_classification()
    


Predicting classes...

Predictions:
Sentence: The new smartphone features an AI-powered camera and improved battery life.
True class: Technology
Predicted class: Health

Sentence: The team scored a last-minute goal to win the championship.
True class: Sports
Predicted class: Health


Class probabilities:
Sentence: The new smartphone features an AI-powered camera and improved battery life.
  Technology: 0.2088
  Sports: 0.1837
  Politics: 0.1790
  Entertainment: 0.2088
  Health: 0.2198

Sentence: The team scored a last-minute goal to win the championship.
  Technology: 0.2084
  Sports: 0.1836
  Politics: 0.1791
  Entertainment: 0.2088
  Health: 0.2201



### Task B: Sentiment Analysis

In [8]:

def sentiment_analysis():
    """Run the inference for the sentiment analysis model with sentence embedding backbone"""
    # Define sentiment analysis classes
    class_names = ["Negative", "Neutral", "Positive"]
    
    # Sample data with varied sentiment
    sample_sentences = [
        "This product is absolutely terrible and completely failed to meet my expectations.",
        "The service was okay, but nothing special to write home about.",
        "I'm absolutely thrilled with my purchase! Best decision I've made all year.",
        "The customer support experience was frustrating and completely unhelpful.",
        "The movie was fairly standard, pretty much what you'd expect from this genre.",
        "The restaurant exceeded all my expectations - incredible food and impeccable service!",
        "I regret spending money on this disappointing product.",
        "The conference covered the topics that were advertised but lacked depth.",
        "The hotel staff went above and beyond to make our stay memorable and special.",
        "This app constantly crashes and has wasted hours of my time.",
        "The book contains useful information presented in a straightforward manner.",
        "The concert was an amazing experience that I'll remember for years to come."
    ]
    
    # Sample labels (0=Negative, 1=Neutral, 2=Positive)
    sample_labels = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
    
    # Initialize the sentence transformer
    sentence_transformer = SentenceTransformer(
        model_name='bert-base-uncased',
        pooling_strategy='cls',
        normalize_embeddings=True
    )
    
    # Initialize the classifier
    classifier = SentenceClassifier(
        sentence_transformer=sentence_transformer,
        num_classes=len(class_names),
        hidden_layers=[256, 128],  # Two hidden layers
        freeze_transformer=True    # Freeze transformer weights
    )

    # Make predictions
    print("\nPredicting classes...")
    predictions = classifier.classify(sample_sentences[:2], class_names=class_names)
    
    # Print predictions
    print("\nPredictions:")
    for i, (sentence, true_class, pred_class) in enumerate(zip(sample_sentences[:2], 
                                                            [class_names[i] for i in sample_labels[:2]], 
                                                            predictions)):
        print(f"Sentence: {sentence}")
        print(f"True class: {true_class}")
        print(f"Predicted class: {pred_class}")
        print()
    
    # Get classification probabilities
    print("\nClass probabilities:")
    probs = classifier.pred_probs(sample_sentences[:2])  # Just show first two for brevity
    for i, (sentence, prob) in enumerate(zip(sample_sentences[:2], probs)):
        print(f"Sentence: {sentence}")
        for j, class_name in enumerate(class_names):
            print(f"  {class_name}: {prob[j]:.4f}")
        print()

if __name__ == "__main__":
    sentiment_analysis()
    


Predicting classes...

Predictions:
Sentence: This product is absolutely terrible and completely failed to meet my expectations.
True class: Negative
Predicted class: Neutral

Sentence: The service was okay, but nothing special to write home about.
True class: Neutral
Predicted class: Neutral


Class probabilities:
Sentence: This product is absolutely terrible and completely failed to meet my expectations.
  Negative: 0.3247
  Neutral: 0.3494
  Positive: 0.3259

Sentence: The service was okay, but nothing special to write home about.
  Negative: 0.3245
  Neutral: 0.3491
  Positive: 0.3264



### Multi-Task Learning Expansion (Both Classification and Sentiment Analysis in one model)

In [9]:
# Define a sample classification task
cls_classes = ["Technology", "Sports", "Politics", "Entertainment", "Health"]
sentiment_classes = ["Negative", "Neutral", "Positive"]

mt_sentences = [
    # Technology - Negative
    "The new smartphone suffers from overheating issues and frequent app crashes.",
    # Sports - Neutral
    "The team played their final match of the season, ending in a 1-1 draw.",
    # Politics - Positive
    "The senator was praised for introducing an innovative bill to tackle climate change.",
    # Entertainment - Negative
    "The movie failed to impress critics, receiving mostly poor reviews.",
    # Health - Neutral
    "Regular exercise and a balanced diet are commonly recommended by health professionals.",
    # Technology - Positive
    "The software update significantly boosted performance and added exciting new features.",
    # Sports - Negative
    "The athlete was disqualified after a false start in the 100-meter sprint.",
    # Politics - Neutral
    "The president is scheduled to speak about economic policy at tomorrow’s event.",
    # Entertainment - Positive
    "The concert was a spectacular event, thrilling fans and selling out in minutes.",
    # Health - Negative
    "The study highlighted how poor sleep habits can impair brain function over time."
]

# Sample labels (indices of class_names)
cls_labels = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
sentiment_labels = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0]


In [10]:
class MultiTaskSentenceTransformer(nn.Module):
    def __init__(self, sentence_transformer, num_classes, num_sentiment_classes=3, hidden_layers=None, freeze_transformer=True):
        """
        Initialize a classifier that uses sentence embeddings.
        
        Args:
            sentence_transformer: SentenceTransformer model for creating embeddings
            num_classes: Number of output classes for classification
            num_sentiment_classes: Number of output classes for sentiment analysis
            hidden_layers: List of hidden layer sizes (if None, only a single classification layer is used)
            freeze_transformer: Whether to freeze the transformer part during training
        """
        super(MultiTaskSentenceTransformer, self).__init__()
        self.sentence_transformer = sentence_transformer
        if freeze_transformer:
            for param in self.sentence_transformer.transformer.parameters():
                param.requires_grad = False
        embedding_dim = self.sentence_transformer.embedding_dim

        # Classification head
        if hidden_layers is None:
            self.classification_head = nn.Linear(embedding_dim, num_classes)
            self.sentiment_head = nn.Linear(embedding_dim, num_sentiment_classes)
        else:
            layers = []
            input_dim = embedding_dim
            for hidden_dim in hidden_layers:
                layers.append(nn.Linear(input_dim, hidden_dim))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(0.1))
                input_dim = hidden_dim
            cls_head = layers + [nn.Linear(input_dim, num_classes)]
            sent_head = layers + [nn.Linear(input_dim, num_sentiment_classes)]
            self.classification_head = nn.Sequential(*cls_head)
            self.sentiment_head = nn.Sequential(*sent_head)

    def forward(self, input_ids, attention_mask):
        """Multitask model forward pass"""
        embeddings = self.sentence_transformer(input_ids, attention_mask)
        class_logits = self.classification_head(embeddings)
        sentiment_logits = self.sentiment_head(embeddings)
        return class_logits, sentiment_logits

# Unified inference function
def multitask_inference(model, sentences, class_names=None, sentiment_names=None, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
        Run unified inference to ouptut both the classification and sentiment analysis predictions.
        
        Args:
            model: The Multi-Task transformer model
            sentences: The sentences to run inference over
            class_names: The class names for classification (in order of the class indices)
            sentiment_names: The class names for sentiment analysis (in order of the sentiment class indices)
            device: torch device to run model inference on (GPU if cuda is available, else CPU)
        """

    model.to(device)
    model.eval()
    predictions = []

    for sentence in sentences:
        inputs = model.sentence_transformer.tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            class_logits, sentiment_logits = model(inputs['input_ids'], inputs['attention_mask'])

            # Get predicted class
            class_pred_idx = torch.argmax(class_logits, dim=1).item()
            sentiment_pred_idx = torch.argmax(sentiment_logits, dim=1).item()

            # Map to class names if provided
            class_pred = class_names[class_pred_idx] if class_names is not None else class_pred_idx
            sentiment_pred = sentiment_names[sentiment_pred_idx] if sentiment_names is not None else sentiment_pred_idx

        predictions.append({
            'sentence': sentence,
            'classification': class_pred,
            'sentiment': sentiment_pred,
            'classification_probs': F.softmax(class_logits, dim=1).cpu().numpy()[0],
            'sentiment_probs': F.softmax(sentiment_logits, dim=1).cpu().numpy()[0]
        })

    return predictions


In [11]:
# Initialize the shared sentence transformer
sentence_transformer = SentenceTransformer(
    model_name='bert-base-uncased',
    pooling_strategy='cls',
    normalize_embeddings=True
)

# Instantiate the multi-task model
mt_model = MultiTaskSentenceTransformer(
    sentence_transformer=sentence_transformer,
    num_classes=len(cls_classes),  # for main classification task
    num_sentiment_classes=len(sentiment_classes),  # for sentiment analysis
    hidden_layers=[256, 128],
    freeze_transformer=True
)

# Perform multi-task inference
results = multitask_inference(
    mt_model,
    mt_sentences,
    class_names=cls_classes,
    sentiment_names=sentiment_classes
)

# No training -> Just inference
for res in results:
    print(f"Sentence: {res['sentence']}")
    print(f"Predicted class: {res['classification']} with probs {res['classification_probs']}")
    print(f"Predicted sentiment: {res['sentiment']} with probs {res['sentiment_probs']}")
    print()


Sentence: The new smartphone suffers from overheating issues and frequent app crashes.
Predicted class: Health with probs [0.20356297 0.18196648 0.20348822 0.20161143 0.2093709 ]
Predicted sentiment: Negative with probs [0.3638863  0.29467952 0.34143424]

Sentence: The team played their final match of the season, ending in a 1-1 draw.
Predicted class: Health with probs [0.20371853 0.18196344 0.20314692 0.2020812  0.20908993]
Predicted sentiment: Negative with probs [0.3651609 0.2940458 0.3407933]

Sentence: The senator was praised for introducing an innovative bill to tackle climate change.
Predicted class: Health with probs [0.20351812 0.18172985 0.20337825 0.20194069 0.20943314]
Predicted sentiment: Negative with probs [0.3644743  0.29452685 0.34099886]

Sentence: The movie failed to impress critics, receiving mostly poor reviews.
Predicted class: Health with probs [0.20330909 0.18147889 0.20359148 0.2022632  0.20935728]
Predicted sentiment: Negative with probs [0.36463785 0.29530233

## Task 4: Training the Multi-Task Model

In [12]:
def train_multitask_model(model, train_data, train_labels, train_sentiments, 
                         epochs=3, batch_size=8, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Train a multi-task model with both classification and sentiment analysis
    
    Args:
        model: MultiTaskSentenceTransformer model
        train_data: List of training sentences
        train_labels: List of training labels (numeric indices)
        train_sentiments: List of sentiment labels (numeric indices)
        epochs: Number of training epochs
        batch_size: Batch size for training
        device: Device to use for training
        
    Returns:
        Training loss history for both tasks
    """
    model.to(device)
    model.train()
    
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    classification_criterion = nn.CrossEntropyLoss()
    sentiment_criterion = nn.CrossEntropyLoss()
    
    labels_tensor = torch.tensor(train_labels, dtype=torch.long, device=device)
    sentiments_tensor = torch.tensor(train_sentiments, dtype=torch.long, device=device)
    
    loss_history = {'classification': [], 'sentiment': []}
    
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_class_loss = 0
        epoch_sent_loss = 0
        
        for i in range(0, len(train_data), batch_size):
            batch_sentences = train_data[i:i+batch_size]
            batch_labels = labels_tensor[i:i+batch_size]
            batch_sentiments = sentiments_tensor[i:i+batch_size]
            
            inputs = model.sentence_transformer.tokenizer(
                batch_sentences, padding=True, truncation=True, 
                return_tensors="pt", max_length=512
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            optimizer.zero_grad()
            
            class_logits, sentiment_logits = model(inputs['input_ids'], inputs['attention_mask'])
            
            class_loss = classification_criterion(class_logits, batch_labels)
            sent_loss = sentiment_criterion(sentiment_logits, batch_sentiments)
            total_loss = class_loss + sent_loss
            
            total_loss.backward()
            optimizer.step()
            
            epoch_class_loss += class_loss.item()
            epoch_sent_loss += sent_loss.item()
            
        avg_class_loss = epoch_class_loss / (len(train_data) // batch_size)
        avg_sent_loss = epoch_sent_loss / (len(train_data) // batch_size)
        
        loss_history['classification'].append(avg_class_loss)
        loss_history['sentiment'].append(avg_sent_loss)
        
        print(f"Classification loss: {avg_class_loss:.4f} | Sentiment loss: {avg_sent_loss:.4f}")
        
    return loss_history


In [13]:
def evaluate_multitask_model(model, test_data, test_labels, test_sentiments,
                            class_names=None, sentiment_names=None,
                            device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    Evaluate the multi-task model on both classification and sentiment analysis
    
    Args:
        model: MultiTaskSentenceTransformer model
        test_data: List of test sentences
        test_labels: List of test labels (numeric indices)
        test_sentiments: List of test sentiment labels (numeric indices)
        class_names: List of class names for classification
        sentiment_names: List of sentiment class names
        device: Device to use for evaluation
        
    Returns:
        Tuple of (classification_report, sentiment_report)
    """
    model.to(device)
    model.eval()
    
    all_class_preds = []
    all_sent_preds = []
    
    with torch.no_grad():
        for sentence in test_data:
            # Tokenize single sentence
            inputs = model.sentence_transformer.tokenizer(
                sentence, padding=True, truncation=True, 
                return_tensors="pt", max_length=512
            )
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # Get predictions
            class_logits, sentiment_logits = model(inputs['input_ids'], inputs['attention_mask'])
            
            # Convert to predictions
            class_pred = torch.argmax(class_logits, dim=1).item()
            sent_pred = torch.argmax(sentiment_logits, dim=1).item()
            
            all_class_preds.append(class_pred)
            all_sent_preds.append(sent_pred)

    # Print predictions
    print("\nPredictions:")
    for i, (sentence, true_label, true_sent, pred_label, pred_sent) in enumerate(zip(test_data,
                                                            [class_names[i] for i in test_labels],
                                                            [sentiment_names[i] for i in test_sentiments],
                                                            [class_names[i] for i in all_class_preds],
                                                            [sentiment_names[i] for i in all_sent_preds])):
        print(f"Sentence: {sentence}")
        print(f"True class: {true_label}")
        print(f"True sentiment: {true_sent}")
        print(f"Predicted class: {pred_label}")
        print(f"Predicted sentiment: {pred_sent}")
        print()
    
    # Generate classification reports
    class_report = classification_report(
        test_labels, all_class_preds, target_names=class_names, zero_division=0
    )
    
    sent_report = classification_report(
        test_sentiments, all_sent_preds, target_names=sentiment_names, zero_division=0
    )
    
    return class_report, sent_report


In [14]:
def train_and_test_multitask():
    # Classification task setup
    class_names = ["Technology", "Sports", "Politics", "Entertainment", "Health"]
    sentiment_names = ["Negative", "Neutral", "Positive"]
    
    # Sample data with both tasks
    train_sentences = [
        # Technology - Negative
        "The new smartphone suffers from overheating issues and frequent app crashes.",
        # Sports - Neutral
        "The team played their final match of the season, ending in a 1-1 draw.",
        # Politics - Positive
        "The senator was praised for introducing an innovative bill to tackle climate change.",
        # Entertainment - Negative
        "The movie failed to impress critics, receiving mostly poor reviews.",
        # Health - Neutral
        "Regular exercise and a balanced diet are commonly recommended by health professionals.",
        # Technology - Positive
        "The software update significantly boosted performance and added exciting new features.",
        # Sports - Negative
        "The athlete was disqualified after a false start in the 100-meter sprint.",
        # Politics - Neutral
        "The president is scheduled to speak about economic policy at tomorrow’s event.",
        # Entertainment - Positive
        "The concert was a spectacular event, thrilling fans and selling out in minutes.",
        # Health - Negative
        "The study highlighted how poor sleep habits can impair brain function over time."
    ]
    
    test_sentences = [
        "The new smartphone suffers from poor battery life.",
        "The basketball team played their final match of the season, and it ended in a draw.",
        "The president was praised for introducing a new bill to tackle climate change.",
        "The movie was disappointing and received poor reviews from critics and audiences alike.",
        "Regular exercise and a good sleep schedule are essential for maintaining good health.",
        "The software update is amazing!",
        "I was sad to see the athlete from Jamaica named Usain Bolt getting disqualified.",
        "The president will deliver a speech on foreign policy.",
        "The tickets for the basketball game sold out within minutes and the game was thrilling to watch.",
        "The study found a strong correlation between a poor diet and low lifespan."
    ]
    
    # Train Labels
    train_cls_labels = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
    train_sentiment_labels = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0]
    
    # Test Labels
    test_cls_labels = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
    test_sentiment_labels = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0]
    
    # Initialize the model
    sentence_transformer = SentenceTransformer(
        model_name='bert-base-uncased',
        pooling_strategy='cls',
        normalize_embeddings=True
    )
    
    model = MultiTaskSentenceTransformer(
        sentence_transformer=sentence_transformer,
        num_classes=len(class_names),
        num_sentiment_classes=len(sentiment_names),
        hidden_layers=[256, 128],
        freeze_transformer=True
    )
    
    # Train the model
    print("Training multi-task model...")
    train_multitask_model(
        model, train_sentences, train_cls_labels, train_sentiment_labels,
        epochs=20, batch_size=4
    )
    
    # Evaluate
    print("\nEvaluation:")
    class_report, sent_report = evaluate_multitask_model(
        model, test_sentences, test_cls_labels, test_sentiment_labels,
        class_names, sentiment_names
    )
    
    print("Classification Report:")
    print(class_report)
    
    print("\nSentiment Analysis Report:")
    print(sent_report)


if __name__ == "__main__":
    train_and_test_multitask()


Training multi-task model...
Epoch 1/20
Classification loss: 2.4031 | Sentiment loss: 1.6439
Epoch 2/20
Classification loss: 2.3907 | Sentiment loss: 1.6302
Epoch 3/20
Classification loss: 2.3827 | Sentiment loss: 1.6265
Epoch 4/20
Classification loss: 2.3759 | Sentiment loss: 1.6183
Epoch 5/20
Classification loss: 2.3633 | Sentiment loss: 1.6114
Epoch 6/20
Classification loss: 2.3464 | Sentiment loss: 1.6009
Epoch 7/20
Classification loss: 2.3326 | Sentiment loss: 1.5852
Epoch 8/20
Classification loss: 2.3032 | Sentiment loss: 1.5826
Epoch 9/20
Classification loss: 2.2906 | Sentiment loss: 1.5559
Epoch 10/20
Classification loss: 2.2567 | Sentiment loss: 1.5363
Epoch 11/20
Classification loss: 2.2240 | Sentiment loss: 1.4894
Epoch 12/20
Classification loss: 2.1588 | Sentiment loss: 1.4507
Epoch 13/20
Classification loss: 2.1475 | Sentiment loss: 1.4748
Epoch 14/20
Classification loss: 2.0770 | Sentiment loss: 1.3541
Epoch 15/20
Classification loss: 2.0186 | Sentiment loss: 1.3449
Epoch

### Further multi-task training modifications discussed in the write-up!